In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import TFAutoModel, Dense
from sklearn.preprocessing import LabelEncoder


In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D, Lambda
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tensorflow as tf

data = pd.read_csv('/content/drive/MyDrive/Emotion_final.csv')
texts = data['Text'].tolist()
labels = data['Emotion'].tolist()

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)

model_name = "michellejieli/emotion_text_classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels).tolist()
test_labels_encoded = label_encoder.transform(test_labels).tolist()

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels_encoded
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels_encoded
))

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

base_model = TFAutoModel.from_pretrained(model_name, from_pt=True)
model = tf.keras.Sequential([
    base_model,
    Lambda(lambda seqs: seqs[0]),
    GlobalAveragePooling1D(),
    Dense(6)
])

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


1073/1073 [==============================] - 242s 198ms/step - loss: 0.3113 - accuracy: 0.8918
Epoch 2/3
1073/1073 [==============================] - 177s 165ms/step - loss: 0.1519 - accuracy: 0.9349
Epoch 3/3
1073/1073 [==============================] - 176s 164ms/step - loss: 0.1271 - accuracy: 0.9420


In [ ]:
model.evaluate(test_dataset.batch(16))



In [ ]:
model.save("/content/drive/MyDrive/emotion_preds")
tokenizer.save_pretrained("/content/drive/MyDrive/emotion_preds")



('/content/drive/MyDrive/emotion_preds/tokenizer_config.json',
 '/content/drive/MyDrive/emotion_preds/special_tokens_map.json',
 '/content/drive/MyDrive/emotion_preds/vocab.json',
 '/content/drive/MyDrive/emotion_preds/merges.txt',
 '/content/drive/MyDrive/emotion_preds/added_tokens.json',
 '/content/drive/MyDrive/emotion_preds/tokenizer.json')